# Compare radar precipitation data to era5 and station data (time series analysis)

Radar data retrieved from the [CEDA Archive](https://catalogue.ceda.ac.uk/uuid/27dd6ffba67f667a18c62de5c3456350) compared to era5 has much higher spatial resolution (1km vs 10km) and temporal resolution (5min vs 1hour). However, accuraccy is crucial for this project. That is why we compare radar precipitation data from era5 and weather stations to decide if it is more precise than previously used era5 data.

We plot:

    - time series station vs era5
    - time series comparing radar, weather station and era5 precipitation values including confusion matrix

Radar data seems to inhibit systematic uncertainties especially with rain that is not detected by weather stations with an overall increased inaccuracy compared to era5 data. In conclusion, we abstain from using radar data for now (see also notebook radar_era5_station_comparison_maps for comparison on maps level between different sources of precipitation).

In [ ]:
# allows update of external libraries without need to reload package
%load_ext autoreload
%autoreload 2

In [ ]:
import sys

LIBRARY_PATH = "/p/home/jusers/ehlert1/juwels/a2/src/"
sys.path.append(LIBRARY_PATH)

import os
import re
import glob
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import logging
import pathlib
import datetime
import tqdm
import xarray
import rioxarray
import convertbng
import pyproj
import h5py
import itertools
import functools
import collections
import plotly.express

logging.basicConfig(level=logging.INFO)


import a2.utils
import a2.dataset
import a2.plotting

In [ ]:
FOLDER_DATA = a2.utils.file_handling.get_folder_data()
FOLDER_TWEETS = FOLDER_DATA / "tweets/"
FILE_TWEETS = (
    FOLDER_TWEETS
    / "2017_2020_tweets_rain_sun_vocab_emojis_locations_bba_Tp_era5_no_bots_normalized_filtered_weather_stations_fix_predicted_simpledeberta_radar.nc"
)
FOLDER_WEATHER_STATIONS = FOLDER_DATA / "weather_stations/"
FILE_WEATHER_STATIONS = FOLDER_WEATHER_STATIONS / "weather_stations_hourly_rainfall_uk_2017-2020_station_number.csv"

FOLDER_ERA5 = FOLDER_DATA / "precipitation/"
FILE_ERA5 = FOLDER_ERA5 / "ds_prec_era5_uk_2017-2020_decum.nc"

FOLDER_RADAR_DAPCEDA = FOLDER_DATA / "precipitation/radar/"

In [ ]:
ds_era = xarray.load_dataset(FILE_ERA5)
ds_era["tp_h_mm"] = ds_era["tp_h"] * 1e3

In [ ]:
df_stations = a2.dataset.load_dataset.load_weather_stations(FILE_WEATHER_STATIONS)
df_stations = df_stations.reset_index()

In [ ]:
a2.plotting.histograms.plot_histogram(df_stations.station_number.value_counts().values)

In [ ]:
ds_era.sel(time_half=np.datetime64("2017-01-01T00:30:00"), longitude=[-2, -4], latitude=[52, 54])

In [ ]:
fig, ax = a2.plotting.utils_plotting.create_figure_axes(figure_size=(20, 8))
ax.axhline(0.1, color="r")
df_sel = a2.dataset.stations.get_dataframe_from_station_number(df_stations, 21, key_station_number="station_number")
time_range = [np.datetime64("2020-02-01T00:30:00"), np.datetime64("2020-02-29T23:30:00")]
ylim = [0, 3]
latitude, longitude = df_sel["latitude"].values[0], df_sel["longitude"].values[0]
latitude, longitude = np.round(latitude, 1), np.round(longitude, 1)
print(latitude, longitude)
a2.plotting.timeseries.plot_timeseries(
    "ob_end_time", "prcp_amt", df_sel, ax=ax, xlim=time_range, label="station", ylim=ylim
)
ds_era.sel(time_half=slice(*time_range), longitude=longitude, latitude=latitude)["tp_h_mm"].plot(
    linewidth=0.3, ax=ax, marker=".", label="era5", ylim=ylim
)
ax.legend()

In [ ]:
fig, ax = a2.plotting.utils_plotting.create_figure_axes(figure_size=(20, 8), aspect="equal")
ds_era.sel(time_half=np.datetime64("2017-01-01T00:30:00"))["tp_h_mm"].plot(ax=ax)
ax.scatter(longitude, latitude)

In [ ]:
fig, ax = a2.plotting.utils_plotting.create_figure_axes(figure_size=(20, 8))
ax.axhline(0.1, color="r")
df_sel = a2.dataset.stations.get_dataframe_from_station_number(df_stations, 12, key_station_number="station_number")
ylim = [0, 3]
latitude, longitude = df_sel["latitude"].values[0], df_sel["longitude"].values[0]
latitude, longitude = np.round(latitude, 1), np.round(longitude, 1)
print(latitude, longitude)
a2.plotting.timeseries.plot_timeseries(
    "ob_end_time", "prcp_amt", df_sel, ax=ax, xlim=time_range, label="station", ylim=ylim
)
# a2.plotting.timeseries.plot_timeseries(time, tp[0], ax=ax, xlim=time_range, label='radar', ylim=ylim)
ds_era.sel(time_half=slice(*time_range), longitude=longitude, latitude=latitude)["tp_h_mm"].plot(
    linewidth=0.3, ax=ax, marker=".", label="era5", ylim=ylim
)
ax.legend()

In [ ]:
time_range = [np.datetime64("2018-04-01T00:00:00"), np.datetime64("2018-04-04T00:00:00")]

In [ ]:
latitudes_station = []
longitudes_station = []
for station_number in range(int(df_stations.station_number.max()) + 1):
    mask_station = df_stations.station_number == station_number
    longitude = df_stations.loc[mask_station, "longitude"].values[0]
    latitude = df_stations.loc[mask_station, "latitude"].values[0]
    latitudes_station.append(latitude)
    longitudes_station.append(longitude)

time, tp = a2.dataset.radar.time_series_from_files(
    *time_range,
    longitudes_station,
    latitudes_station,
    time_delta=1,
    path_to_dapceda=FOLDER_RADAR_DAPCEDA,
    time_delta_units="h",
)

In [ ]:
# fig, ax = a2.plotting.utils_plotting.create_figure_axes(figure_size=(20, 8))
n_rows = 40
n_cols = 3
figsize = (20, n_rows * 8)
fig, axes, axes_colorbar = a2.plotting.utils_plotting.create_axes_grid(
    n_cols=n_cols,
    n_rows=n_rows,
    figure_size=figsize,
    colorbar_width=0.02,
    spacing_x=0.01,
    spacing_y=0.003,
    colorbar_skip_row_col=[[i, 0] for i in range(n_rows)],
)


def plot_single(ax, ax2, ax3, station_number, time, tp, axes_colorbar0, axes_colorbar1):
    time_radar = time[:]
    ax.axhline(0.1, color="r")
    df_sel = a2.dataset.stations.get_dataframe_from_station_number(
        df_stations, station_number, key_station_number="station_number"
    )
    ylim = [0, 3]
    latitude, longitude = df_sel["latitude"].values[0], df_sel["longitude"].values[0]
    latitude, longitude = np.round(latitude, 1), np.round(longitude, 1)
    # print(latitude, longitude)
    a2.plotting.timeseries.plot_timeseries(
        "ob_end_time", "prcp_amt", df_sel, ax=ax, xlim=time_range, label="station", ylim=ylim
    )  # , linewidth=0.3)
    tp_radar = tp[:, station_number, 0]
    a2.plotting.timeseries.plot_timeseries(
        time,
        tp_radar,
        ax=ax,
        xlim=time_range,
        label="radar",
        ylim=ylim,
        color="black",
        alpha=0.6,
        marker=".",
        linestyle="",
    )
    # a2.plotting.timeseries.plot_timeseries(time, tp[0], ax=ax, xlim=time_range, label='radar', ylim=ylim)
    tp_era5 = ds_era.sel(time_half=slice(*time_range), longitude=longitude, latitude=latitude)["tp_h_mm"].values
    time_era5 = ds_era.sel(time_half=slice(*time_range), longitude=longitude, latitude=latitude)["time_half"].values
    t = df_sel.ob_end_time.values
    mask_time_station = np.logical_and(t >= time_range[0], t <= time_range[1])
    time_station = t[mask_time_station]
    tp_station = df_sel.prcp_amt.values[mask_time_station]

    def get_raining(time, tp, threshold):
        mask_sort = np.argsort(time)
        time_sorted, tp_sorted = time[mask_sort], tp[mask_sort]
        rain = np.array(tp_sorted >= threshold, dtype=int)
        return time_sorted, rain

    def plot_confusion_matrix(ax, x, y, label_x, label_y, ax_colorbar):
        a2.plotting.histograms.plot_histogram_2d(
            x,
            y,
            label_x=label_x,
            label_y=label_y,
            ax=ax,
            marginal_x=None,
            marginal_y=None,
            ax_colorbar=ax_colorbar,
            n_bins=2,
            overplot_values=True,
        )

    threshold = 0.1
    time_station, tp_station = get_raining(time_station, tp_station, threshold)
    time_era5, tp_era5 = get_raining(time_era5, tp_era5, threshold)
    time_radar, tp_radar = get_raining(time_radar, tp_radar, threshold)
    time_station, tp_station = time_station[:-1], tp_station[:-1]
    time_radar, tp_radar = time_radar[:-1], tp_radar[:-1]
    if tp_station.shape != tp_era5.shape:
        return
    plot_confusion_matrix(ax2, tp_station, tp_era5, "station", "era5", ax_colorbar=axes_colorbar0)
    plot_confusion_matrix(ax3, tp_station, tp_radar, "station", "radar", ax_colorbar=axes_colorbar1)
    ds_era.sel(time_half=slice(*time_range), longitude=longitude, latitude=latitude)["tp_h_mm"].plot(
        ax=ax, label="era5", ylim=ylim
    )  # , linewidth=0.3)
    ax.legend()


for i_row in range(n_rows):
    # for i_col in range(n_cols):
    #     ax = axes[i_row, i_col]
    i_plot = i_row + i_row * n_cols
    plot_single(*axes[i_row, :], i_row, time, tp, axes_colorbar[i_row][1], axes_colorbar[i_row][2])